# Obtaining structured web content and programmatic access

In this notebook we will learn how to make use of readily structured data through dedicated application programming interfaces (APIs), how to authenticate and how to properly design requests (also called "payload") in order to retrieve large datasets.

**Advantages** of APIs are that
- access is legal and in most cases clearly and transparently regulated (e.g. 10,000 calls per day)
- structuring through `requests` and `BeautifulSoup` not required
- Python packages that simplify server-client interaction are available

**Disadvantages** of APIs are that
- we have to learn how APIs work and how we should interact with them (each API has some peculiarities and documentation is usually good, but sometimes not so...)
- authentification may be required and access may not be free of charge 

We will
- obtain data of a public statistical office such as the IMF or World Bank through the `pandas-datareader`
- directly obtain a (ranking) table from a website such the [World Cube Association](https://www.worldcubeassociation.org/results/rankings/333/single)
- learn how to use the Destatis/GENESIS Online service and API
- learn how to use the Twitter API (in particular [Tweepy](https://www.tweepy.org/), a Python library for the Twitter API) and retrieve Tweets with GeoTags (i.e. coordinates) subject to specified geography and search terms
- conduct some small analyses and visualise the results appropriately

In [ ]:
import pandas as pd

In [ ]:
!pip install pandas-datareader --upgrade

## Directly parsing `table` objects from HTML

In [ ]:
# Recall: %whos shows you the variables in name space

Which nationality appears most frequently in the World Cube Association's ranking?

Which nationality needed, on average, the **lowest** amount of time to solve a 3x3x3 cube? Sort the output in ascending order.

How many possible states/permutations, starting from the solved state, can a 3x3x3 Rubik's cube have? 

Hints:
1. The centre squares are fixed (a plane rotation around these squares doesn't change the cube's state)
2. There are eight corner pieces (with three colors on the side) and twelve edge pieces (with two colors on the side) which all revolve around the centre pieces
3. There are six different colors
4. We only look at "legal" states, i.e. those that can only be realised without assembling the cube (and therefore not violating Hint 1)

Which result entry in the World Ranking table is the most recent one? Which one is the oldest one? Be as precise as possible! (Hint: You may have to combine your knowledge from scraping HTML files.)

Compute the expected value of `Result` conditional on `Nationality = 'Germany'`. Are the German contestants statistically significantly faster/slower in solving the cube than other contestants, based on `Nationality`? Does statistical significance change if you use robust standard errors?

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm

## Spatial libraries 
> (Note to myself: do GENESIS API first!)

The installation procedure of spatial libraries for Python (on Windows) can be quite tedious but [this answer](https://stackoverflow.com/questions/51095970/install-python-geopandas-failed/51560940#51560940) on Stackoverflow (make sure to upvote ;)) and [this detailed instruction](https://geoffboeing.com/2014/09/using-geopandas-windows/) make it straight forward. You can also find the required wheels for Python 3.8 and 64-bit for offline `pip install` in this notebook's [repository](https://github.com/gerwolf/webscraping-workshop/tree/main/DataFrames%20and%20APIs). After this, you can simply `pip install geopandas`.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]

## Destatis/GENESIS Online
The GENESIS API is the web interface service by the Federal Statistical Office of Germany and is a good place to start learning how to interact programmatically with a server.

There is a [comprehensive description/introduction](https://www-genesis.destatis.de/genesis/misc/GENESIS-Webservices_Einfuehrung.pdf) on the service, unfortunately in German only. To display the PDF inside Jupyter Notebook in Chrome you may have to enable the [PDF Viewer extension](https://chrome.google.com/webstore/detail/pdf-viewer/oemmndcbldboiebfnladdacbdfmadadm?utm_source=chrome-ntp-icon).

In [ ]:
from IPython.display import IFrame, display
filepath = "https://www-genesis.destatis.de/genesis/misc/GENESIS-Webservices_Einfuehrung.pdf"
IFrame(filepath, width=980, height=800)

Read about the `whoami` method (in Section 2.2). Do you have to authenticate? What does it return? How would you send a `request`?

In [ ]:
import genesis_config
import requests

Read about the `logincheck` method (in Section 2.2). Do you have to authenticate? What does it return? Construct a request object using string formatting, send a `request` (in English language) and print the request's status. What type is the response's `text` attribute?

Now that we have a working connection to the GENESIS Online API we want to directly obtain an economic indicator, the private sector's savings rate on a quarterly basis, for instance. This `data` is usually stored in a `table` somewhere in the depths of a data warehouse and it is (unfortunately) necessary to familiarise yourself, at least partially, with the internal server's structure.
1. In the documentation file search for the `tablefile` method (under Section 2.5 Data). When should it be used? What does it return?
2. Which method should you use if you want to directly obtain a `chart`?
3. Which method should you use if you want to directly obtain a regional `map`? Which parameter controls the image's resolution?

4. Login to the [GENESIS Online user interface](https://www-genesis.destatis.de/genesis/online?Menu=Anmeldung#abreadcrumb). Familiarise yourself with the tables' structure and navigate to the National Accounts (at the central level) --> Private sector disposable income and savings at quarterly frequency. Which parameters in the request can you control?
5. Which method would you choose if you want to directly obtain a `table` in some machine readable format, e.g. a `.csv` or `.xlsx` that you can read into `pandas`? Which output `formats` can you choose from? How do you include additional conditions matching particular values?
6. Construct a `request` which contains the following specification:
    - only seasonally and calendar-adjusted values (X13)
    - all available years and quarters
    - output format should be a `.xlsx` file
7. Send the request, but directly through the `pandas.read_excel()` method.

In [ ]:
! pip install openpyxl

## Twitter API


In [ ]:
!pip install python-twitter --upgrade

In [ ]:
import twitter
import twitter_config

api = twitter.Api(consumer_key = twitter_config.api_key ,
                  consumer_secret = twitter_config.api_secret_key,
                  access_token_key = twitter_config.access_token,
                  access_token_secret = twitter_config.access_token_secret,
                  tweet_mode = 'extended',
                  sleep_on_rate_limit = True)

Let us query a free-text-search using the hashtag `#gameofthrones`. Which function would you use? Limit the maximum amount of response elements to 10.

Which type is the output object? Which type have the output object's elements?

In [ ]:
!pip install tweepy --upgrade

In [3]:
import tweepy
import json
import twitter_config

In [13]:
auth = tweepy.OAuthHandler(twitter_config.api_key, twitter_config.api_secret_key)
auth.set_access_token(twitter_config.access_token, twitter_config.access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

`Tweepy` is great for retrieving additional info on Tweets such as GeoLocation (only for those where users have enabled the feature, or not consciously disabled it)
- We will run a small analysis of the spatial distribution of recent Tweets in Germany
- The common finding (also known in the academic and non-academic literature) is that the data quality decreases substantially once we condition those observations on specific keywords
> Please note that Twitter’s search service and, by extension, the Search API is not meant to be an exhaustive source of Tweets. Not all Tweets will be indexed or made available via the search interface. (From the [official Twitter API documentation](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets))

Your task now: familiarise yourself with the [`Twitter ` API](https://developer.twitter.com/en/docs/twitter-api/v1) and [`Tweepy` Python](https://docs.tweepy.org/en/v3.10.0/index.html) implementation.

In [12]:
from IPython.display import IFrame

url = 'https://docs.tweepy.org/en/v3.10.0/index.html'
IFrame(url, width=980, height=400)

1. Which `Tweepy` version have you installed?
2. Which function would you use to send a search query?
3. Which function in `Tweepy` facilitates pagination?
4. Which fields contain [spatial](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/geo) information?
5. Identify the internal country code of `Germany`, i.e. its `id`.
6. How would you [construct a query](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets) that specifies a search term, the language and the number of Tweets shown per page?

Note there are [rate limits](https://developer.twitter.com/en/docs/twitter-api/rate-limits)! For example, include `time.sleep(15 * 60)` in case your rate limit was reached when running more comprehensive searches.

In [39]:
# Example query

header = {"q": "place:fdcd221ac44fa326", "lang": "de", "tweet_mode": "extended", "count": 100}

# Note: "count":100 is the maximum of returned Tweets per request/item in the standard Twitter API
# You can paginate either per item (i.e. individual request) or pages of results 
# https://docs.tweepy.org/en/latest/cursor_tutorial.html
# The Cursor() function ignores the 'count' parameter in the query, the limit is defined in the items() function instead

batch_tweets = tweepy.Cursor(api.search, **header)

noOfSearch = 200

batch_items = batch_tweets.items(noOfSearch)

In [40]:
batch_list = [i._json for i in batch_items] # underscore means 'intended for internal use'

Write a function which takes as input arguments a (not hardcoded) `place_id`, `language` and `tweet_mode` (which you set to "extended" by default such that you query the full, untruncated text content of the Tweets) and returns a **list of dictionaries**. The goal is to obtain a as-complete-as-possible collection of recent Tweets in Germany **without specifying a `key word`**. In order to maximize the number of Tweets you receive, stack the `.Cursor()` object inside of a for-loop. Save the resulting output to your local directory as a `.json` file.

In [ ]:
# Your code















Once you have the raw output, determine the total number of unique Tweets, i.e. `ids`.
You can use a dictionary comprehension (similar to list comprehensions) to achieve that since duplicate keys are not allowed in a dictionary.

In [ ]:
unique = { tweet['id'] : tweet for tweet in bunches }
len(unique)

Identify all Tweets (elements) which value associated with the key `coordinates` is different from `None`. Assign the result to an object `matches`. How does the content of this object compare to the "unfiltered" data container `unique`?

In [ ]:
# Code for matching elements

You should get elements in this format:

```
{1382787792535703552: {'created_at': 'Thu Apr 15 20:07:55 +0000 2021',
  'id': 1382787792535703552,
  'id_str': '1382787792535703552',
  'full_text': '@bennai_m @JoueursDZ @DZfoot Ya sehr gut geschossen Grosse Verantwortung',
  'truncated': False,
  'display_text_range': [29, 72],
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'bennai_m',
     'name': 'MBennai',
     'id': 1157689821969756161,
     'id_str': '1157689821969756161',
     'indices': [0, 9]},
    {'screen_name': 'JoueursDZ',
     'name': 'JDZ Football',
     'id': 1177267711157690371,
     'id_str': '1177267711157690371',
     'indices': [10, 20]},
    {'screen_name': 'DZfoot',
     'name': 'DZfoot',
     'id': 296303972,
     'id_str': '296303972',
     'indices': [21, 28]}],
   'urls': []},
  'metadata': {'iso_language_code': 'de', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'in_reply_to_status_id': 1382781123978223624,
  'in_reply_to_status_id_str': '1382781123978223624',
  'in_reply_to_user_id': 1157689821969756161,
  'in_reply_to_user_id_str': '1157689821969756161',
  'in_reply_to_screen_name': 'bennai_m',
  'user': {'id': 1370800515295100933,
   'id_str': '1370800515295100933',
   'name': 'Tassilli Hoggar',
   'screen_name': 'TassilliHoggar',
   'location': '',
   'description': 'sports',
   'url': None,
   'entities': {'description': {'urls': []}},
   'protected': False,
   'followers_count': 17,
   'friends_count': 113,
   'listed_count': 0,
   'created_at': 'Sat Mar 13 18:15:23 +0000 2021',
   'favourites_count': 427,
   'utc_offset': None,
   'time_zone': None,
   'geo_enabled': True,
   'verified': False,
   'statuses_count': 959,
   'lang': None,
   'contributors_enabled': False,
   'is_translator': False,
   'is_translation_enabled': False,
   'profile_background_color': 'F5F8FA',
   'profile_background_image_url': None,
   'profile_background_image_url_https': None,
   'profile_background_tile': False,
   'profile_image_url': 'http://pbs.twimg.com/profile_images/1370801015902064643/BCSHPcOg_normal.jpg',
   'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1370801015902064643/BCSHPcOg_normal.jpg',
   'profile_link_color': '1DA1F2',
   'profile_sidebar_border_color': 'C0DEED',
   'profile_sidebar_fill_color': 'DDEEF6',
   'profile_text_color': '333333',
   'profile_use_background_image': True,
   'has_extended_profile': True,
   'default_profile': True,
   'default_profile_image': False,
   'following': False,
   'follow_request_sent': False,
   'notifications': False,
   'translator_type': 'none'},
  'geo': None,
  'coordinates': None,
  'place': {'id': '5bcd72da50f0ee77',
   'url': 'https://api.twitter.com/1.1/geo/id/5bcd72da50f0ee77.json',
   'place_type': 'city',
   'name': 'Hamburg',
   'full_name': 'Hamburg, Germany',
   'country_code': 'DE',
   'country': 'Germany',
   'contained_within': [],
   'bounding_box': {'type': 'Polygon',
    'coordinates': [[[8.4201604, 53.395118],
      [10.325199, 53.395118],
      [10.325199, 53.9646546],
      [8.4201604, 53.9646546]]]},
   'attributes': {}},
  'contributors': None,
  'is_quote_status': False,
  'retweet_count': 0,
  'favorite_count': 0,
  'favorited': False,
  'retweeted': False,
  'lang': 'de'}}
  ```

In [ ]:
from shapely.geometry import Point, Polygon

Now we're using a spatial library to process those appropriate data fields. The coordinates are in rectangular form and we would like to collapse them to a single point (e.g. to increase the accuracy). 
- First you need to convert the raw rectangular coordinates into a shape (use `Polygon()`), obtain the centroid from the resulting object and separate it into `longitude` and `latitude`. 
- Then combine the two coordinates to a `Point()` coordinates object.
- Extract the following key-values from the `matches` data container:
    - key
    - timestamp of creation
    - full Tweet text
    - the user's registered name
    - the user's nickname/alias as shown on Twitter
    - the place's ID
    - the place's name
    - the place's country code
    - the place's country name
    - the place's bounding box' coordinates
    - the place's centroid's coordinates (as Shapely object)
    - the Tweet's number of Retweets
    - the Tweet's number of likes/favorites
    - the user's location (e.g. checked in @ Universität Potsdam)
    - the number of user's followers
    - the number of user's friends
    - the registration date of the user's account
    - a complete URL that leads to the original Tweet (Hint: it's a combination of two other fields from above, use string formatting)

The output should have the following format:

```
{'Tweet ID': 1382787792535703552,
 'Created at': Timestamp('2021-04-15 20:07:55+0000', tz='UTC'),
 'Full Text': '@bennai_m @JoueursDZ @DZfoot Ya sehr gut geschossen Grosse Verantwortung',
 'User Name': 'Tassilli Hoggar',
 'User Alias': 'TassilliHoggar',
 'Place ID': '5bcd72da50f0ee77',
 'Place Name': 'Hamburg',
 'Country Code': 'DE',
 'Country Name': 'Germany',
 'Bounding Box': [[8.4201604, 53.395118],
  [10.325199, 53.395118],
  [10.325199, 53.9646546],
  [8.4201604, 53.9646546]],
 'Tweet Coordinates': <shapely.geometry.point.Point at 0x292c8039f10>,
 'Retweet Count': 0,
 'Favorite Count': 0,
 'User Location': '',
 'User Followers': 17,
 'User Friends': 113,
 'Registration Date': 'Sat Mar 13 18:15:23 +0000 2021',
 'Tweet URL': 'https://twitter.com/TassilliHoggar/status/1382787792535703552'}
```

In [ ]:
# Your code for parsing



















In [ ]:
results_df = pd.DataFrame(results_list)
gdf = gpd.GeoDataFrame(results_df, geometry=results_df['Tweet Coordinates'])
gdf.crs = "epsg=4326"

In [ ]:
mapbox_access_token = open("mapbox_token.txt").read()

In [ ]:
# Code for creating the Plotly graph

In [ ]:
! pip install scikit-learn

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
palette = sns.color_palette(None, 11).as_hex()

In [ ]:
# germany_borders = gpd.read_file("Bundeslaender_2016_ew.shp")
# You can find it here: https://opendata-esri-de.opendata.arcgis.com/datasets/b8d0cc7735774bed8e6df1c5410394a4_0?geometry=-31.360%2C46.270%2C52.268%2C55.886

In [ ]:
germany_borders = gpd.read_file("https://opendata.arcgis.com/datasets/b8d0cc7735774bed8e6df1c5410394a4_0.geojson")

In [ ]:
# https://stackoverflow.com/questions/64200595/geopandas-overlay-intersection-returns-zero-rows THANK YOU!

# Multiple solutions: https://gis.stackexchange.com/questions/208546/check-if-a-point-falls-within-a-multipolygon-with-python



How can the clusters and memberships you assigned before be meaningfully interpreted? How many Tweets (as percentage of all per state) were classified into the cluster with `cluster id = 8`? From which state does those cluster members most likely originate from? Confirm your assertion by looking at the map you created above.